In [1]:
from bs4 import BeautifulSoup
from glob import glob
import os
import re

# Where are all those htmls?
html_route = r"C:\Corpora\zelda-wikia2\ZeldaWikia2\zelda.wikia.com\wiki"
# Where should I place the clean versions
clean_html_route = r"C:\Corpora\zelda-wikia2-clean"

omit_files = set()

In [2]:
from cleaner import remove_tags

def pull_content(soup):
    pageHeader = soup.find('header', {"id":"PageHeader"})
    page_title = None
    wikiaMainContent = soup.find('article', {"id":"WikiaMainContent"})
    if pageHeader:
        page_title = pageHeader.find('h1').text
        wikiaMainContent['title'] = page_title
    
    return wikiaMainContent

def clean_article(soup):
    content_only = pull_content(soup) 
    resulting_soup = remove_tags(content_only, tags=['script', 'link', 'style','noscript'], parser="lxml")
    return resulting_soup

In [3]:
# Take files out of the folders (they were generated because they contain backslashes in the filename)

backslash ="%2F"
from shutil import copyfile

main_get_dirs = html_route + r"\*\\"
root_folder = html_route 
for dr in glob(main_get_dirs):
    parts = dr.split("\\")
    for inner_file in glob(dr + "*.html"):
        a = os.path.basename(inner_file)
        final_name = parts[-2]+ backslash + a
        dst = os.path.join(root_folder, final_name)
        copyfile(inner_file, dst)

In [4]:
last_hex = re.compile(".+[0-9a-fA-F]{4}.html")
last_hex_alt = re.compile(".+[0-9a-fA-F]{4}-[0-9]{1,2}.html")

main_dir = html_route + r"\*.html"
original_files = set(glob(main_dir))

for file in glob(main_dir):
    a = os.path.basename(file)
    if last_hex.search(a): # Omit files that are revisions
        omit_files.add(file)
    if last_hex_alt.search(a): # Omit files that are revisions
        omit_files.add(file)
    elif a.startswith("User_"):
        omit_files.add(file)
    elif a.startswith("Talk_"):
        omit_files.add(file)
    elif a.startswith("Template_"):
        omit_files.add(file)
    elif a.startswith("Category_talk_"):
        omit_files.add(file)
    elif a.startswith("Walkthrough_"):
        omit_files.add(file)
    elif a.startswith("Zeldapedia_talk_"):
        omit_files.add(file)
    elif a.startswith("Help_"):
        omit_files.add(file)
    elif a.startswith("Category_Zeldapedians_"):
        omit_files.add(file)
    elif a.startswith("Special_"):
        omit_files.add(file)
    elif a.startswith("File_"):
        omit_files.add(file)
    elif a.startswith("Zeldapedia_"):
        omit_files.add(file)
    elif a.startswith("Forum_"):
        omit_files.add(file)
    elif a.startswith("Message_Wall_"):
        omit_files.add(file)
len(omit_files)

42171

In [5]:
print(len(omit_files))
print(len(original_files))
valuable_files =  original_files - omit_files

42171
50885


In [6]:
files_to_work_with = sorted(list(valuable_files))
output_pattern = clean_html_route + r"\%s"

for file in files_to_work_with:
    a = os.path.basename(file)
    output_file = output_pattern % a
    data = None
    
    with open(file, "r", encoding="utf8") as r:
        data = BeautifulSoup(r, "lxml")
    resulting_soup = clean_article(data)
        
    with open(output_file, "wb") as w:
        w.write(resulting_soup.prettify("utf8"))